# Retrieving NBA data with API, Writing to Postgresql

In [1]:
# download psycopg2
pip install psycopg2-binary

     ---------------------------------------- 1.2/1.2 MB 5.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


## Getting General Data of the Last 20 Games

In [10]:
import pandas as pd
import psycopg2  # PostgreSQL driver
import psycopg2.extras  # For high-performance bulk inserts using execute_values
from nba_api.stats.endpoints import leaguegamefinder  # NBA API endpoint to retrieve team-game data

# ---------------- Options / Configuration ----------------
SEASON = "2024-25"           # NBA season to fetch
MAX_GAMES = 20               # Limit to the last 20 games
DB_HOST = "localhost"        # PostgreSQL host
DB_PORT = 5432               # PostgreSQL port
DB_NAME = "postgres"         # Database name
DB_USER = "postgres"         # DB user
DB_PASSWORD = "1234"         # DB password
DB_TABLE = "teams_plays"     # Table where results will be stored
DB_SCHEMA = "nba"            # Schema name

# ---------------- Database Connection ----------------
conn = psycopg2.connect(
    host=DB_HOST,
    port=DB_PORT,
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD
)

# ---------------- Helper Functions ----------------
def log(msg):
    # Simple wrapper for printing logs
    print(msg)

def fetch_season_team_games(season=SEASON):
    """
    Fetch all team-game data for the specified season using NBA API.
    Returns a DataFrame where each row represents one team in one game.
    """
    log(f"[1] Fetching season data: {season} ...")
    finder = leaguegamefinder.LeagueGameFinder(season_nullable=season, league_id_nullable='00')
    df = finder.get_data_frames()[0]  # First DataFrame contains team-game rows
    log(f"[1] Total rows (team-game): {len(df)}")
    return df

def normalize_basic(df):
    """
    Normalize raw NBA data:
    - Select desired columns
    - Convert GAME_DATE to datetime
    - Determine HOME or AWAY from MATCHUP
    - Calculate opponent points (PTS_OPP) and point difference (POINT_DIFF)
    - Sort and prepare DataFrame for DB insertion
    """
    want_cols = [
        'GAME_ID', 'GAME_DATE', 'SEASON', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME',
        'MATCHUP', 'WL', 'PTS'
    ]
    df = df[[c for c in want_cols if c in df.columns]].copy()
    
    df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])
    
    # Determine Home/Away based on MATCHUP string
    df['HOME_AWAY'] = df['MATCHUP'].apply(
        lambda m: 'Home' if ('vs' in str(m) or 'vs.' in str(m)) 
        else ('Away' if '@' in str(m) else 'Unknown')
    )

    # Compute opponent points (PTS_OPP) and point difference
    tmp = df[['GAME_ID','TEAM_ID','PTS']].copy()
    merged = pd.merge(tmp, tmp, on='GAME_ID', suffixes=('','_opp'))
    merged = merged[merged['TEAM_ID'] != merged['TEAM_ID_opp']]
    merged = merged[['GAME_ID','TEAM_ID','PTS_opp']].rename(columns={'PTS_opp':'PTS_OPP'})
    df = df.merge(merged, on=['GAME_ID','TEAM_ID'], how='left')
    df['POINT_DIFF'] = df['PTS'] - df['PTS_OPP']

    # Keep only DB-ready columns
    df = df[['GAME_ID','GAME_DATE','TEAM_ID','TEAM_ABBREVIATION','TEAM_NAME',
             'MATCHUP','WL','PTS','HOME_AWAY','PTS_OPP','POINT_DIFF']]
    # Sort by date and game_id
    df = df.sort_values(['GAME_DATE','GAME_ID','TEAM_ABBREVIATION']).reset_index(drop=True)
    return df

def upsert_df(df: pd.DataFrame, table: str):
    """
    Bulk insert DataFrame into DB.
    Uses ON CONFLICT to upsert on (GAME_ID, TEAM_ID).
    """
    if df.empty:
        log(f"[WARN] DataFrame is empty, skipping insert.")
        return

    cols = list(df.columns)
    col_str = ",".join(cols)
    update_str = ",".join([f"{c}=EXCLUDED.{c}" for c in cols if c not in ("GAME_ID","TEAM_ID")])
    
    # Prepare SQL for bulk insert with upsert
    sql = f"""
    INSERT INTO {DB_SCHEMA}.{table} ({col_str})
    VALUES %s
    ON CONFLICT (GAME_ID, TEAM_ID) DO UPDATE SET {update_str};
    """
    # Convert DataFrame to list of tuples for execute_values
    data = [tuple(x) for x in df.to_numpy()]
    
    # Execute bulk insert using execute_values for performance
    with conn.cursor() as cur:
        # page_size=500 splits data into chunks of 500 rows
        psycopg2.extras.execute_values(cur, sql, data, page_size=500)
    conn.commit()
    log(f"[OK] {table} table updated with {len(df)} rows.")

# ---------------- Main Execution ----------------
if __name__ == "__main__":
    # Fetch season games
    raw = fetch_season_team_games(SEASON)
    # Normalize and enrich
    basic = normalize_basic(raw)

    # Filter last MAX_GAMES unique games
    unique_games = basic['GAME_ID'].drop_duplicates().tolist()
    last_20_games = unique_games[-MAX_GAMES:]
    basic_last20 = basic[basic['GAME_ID'].isin(last_20_games)].copy()

    # Write to DB
    upsert_df(basic_last20, DB_TABLE)
    log("[✓] Process completed.")

[1] Sezon verisi çekiliyor: 2024-25 ...
[1] Toplam satır (team-game): 2802
[OK] teams_plays tablosuna 40 satır yazıldı.
[✓] İşlem tamamlandı.


## Getting Statistical Data of the Last 20 Matches

In [23]:
# =========================================
# Required packages:
# pip install nba_api pandas tqdm psycopg2
# =========================================

# Import necessary libraries
from nba_api.stats.endpoints import leaguegamefinder, boxscoretraditionalv2
import pandas as pd
import time
from tqdm import tqdm        # Progress bar
import psycopg2             # PostgreSQL database connection

# ---------------- Options / Configuration ----------------
SEASON = "2024-25"           # NBA season to fetch
BATCH_SIZE = 20              # Limit to last 20 games
SLEEP_BETWEEN_CALLS = 1.5    # Delay between API calls (seconds)
MAX_RETRIES = 5              # Maximum retry attempts if API fails
VERBOSE = True               # Enable/disable logging

DB_CONFIG = {                # PostgreSQL connection configuration
    "host":"localhost",
    "port":5432,
    "dbname":"postgres",
    "user":"postgres",
    "password":"1234"
}

SCHEMA = "nba"               # Target schema in DB
TABLE_NAME = "team_games_boxscore"  # Target table for upserts

# ----------------------------------------------------------

# Simple logging function
def log(*args):
    if VERBOSE:
        print(*args)

# ---------------- DB Connection ----------------
# Connect to PostgreSQL using psycopg2
conn = psycopg2.connect(**DB_CONFIG)

# ---------------- Function: Upsert a row into DB ----------------
def upsert_row(row):
    """
    Inserts a row into the target table.
    If the combination (GAME_ID, TEAM_ID) already exists, it updates the existing row.
    """
    cols = list(row.index)
    col_str = ",".join(cols)
    # Build update string for ON CONFLICT
    update_str = ",".join([f"{c}=EXCLUDED.{c}" for c in cols if c not in ("GAME_ID","TEAM_ID")])
    sql = f"""
    INSERT INTO {SCHEMA}.{TABLE_NAME} ({col_str})
    VALUES ({','.join(['%s']*len(cols))})
    ON CONFLICT (GAME_ID, TEAM_ID) DO UPDATE SET {update_str};
    """
    try:
        with conn.cursor() as cur:
            cur.execute(sql, tuple(row.values))  # Execute the SQL with row values
        conn.commit()
        log(f"[OK] Row GAME_ID={row['GAME_ID']} TEAM_ID={row['TEAM_ID']} written")
    except Exception as e:
        conn.rollback()
        log(f"[WARN] Row GAME_ID={row['GAME_ID']} TEAM_ID={row['TEAM_ID']} failed: {e}")

# ---------------- NBA API Functions ----------------

def fetch_basic_games():
    """
    Fetch all basic game data for the specified season from NBA API
    """
    log(f"[1] Fetching season data: {SEASON} ...")
    finder = leaguegamefinder.LeagueGameFinder(season_nullable=SEASON, league_id_nullable='00')
    df = finder.get_data_frames()[0]       # First dataframe contains team-game rows
    df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])  # Ensure GAME_DATE is datetime
    log(f"[1] Total rows (team-games): {len(df)}")
    return df

def normalize_basic(df):
    """
    Normalize the basic games dataframe:
    - Keep only required columns
    - Calculate HOME_AWAY status
    - Calculate opponent points (PTS_OPP) and point difference
    """
    want_cols = [
        'GAME_ID','GAME_DATE','SEASON','TEAM_ID','TEAM_ABBREVIATION','TEAM_NAME',
        'MATCHUP','WL','PTS'
    ]
    # Keep only available columns
    available = [c for c in want_cols if c in df.columns]
    out = df[available].copy()

    # Determine Home/Away from MATCHUP string
    out['HOME_AWAY'] = out['MATCHUP'].apply(
        lambda m: 'Home' if ('vs' in str(m) or 'vs.' in str(m)) else ('Away' if '@' in str(m) else 'Unknown')
    )

    # Calculate opponent points and point difference
    if 'PTS' in out.columns:
        tmp = out[['GAME_ID','TEAM_ID','PTS']].copy()
        merged = pd.merge(tmp,tmp,on='GAME_ID',suffixes=('','_opp'))
        merged = merged[merged['TEAM_ID'] != merged['TEAM_ID_opp']]
        merged = merged[['GAME_ID','TEAM_ID','PTS_opp']].rename(columns={'PTS_opp':'PTS_OPP'})
        out = out.merge(merged,on=['GAME_ID','TEAM_ID'],how='left')
        out['POINT_DIFF'] = out['PTS'] - out['PTS_OPP']
    else:
        out['PTS_OPP'] = None
        out['POINT_DIFF'] = None

    # Sort results by date, game_id, team abbreviation
    return out.sort_values(['GAME_DATE','GAME_ID','TEAM_ABBREVIATION']).reset_index(drop=True)

def fetch_team_boxscore(game_id):
    """
    Fetch team-level boxscore data for a single GAME_ID.
    Returns one row per team with numeric stats aggregated.
    Implements retries with exponential backoff.
    """
    attempt = 0
    sleep_time = SLEEP_BETWEEN_CALLS
    while attempt <= MAX_RETRIES:
        try:
            bs = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_id)
            dfs = bs.get_data_frames()
            team_df = None
            for df in dfs:
                if 'TEAM_ID' in df.columns:
                    team_df = df.copy()
                    break
            if team_df is None:
                raise RuntimeError(f"Game {game_id}: no team-level dataframe found.")
            
            # Keep only desired columns
            keep_cols = [
                'GAME_ID','TEAM_ID','TEAM_ABBREVIATION','PTS','REB','AST','BLK',
                'TOV','FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA','FT_PCT'
            ]
            available = [c for c in keep_cols if c in team_df.columns]
            if 'GAME_ID' not in team_df.columns:
                team_df['GAME_ID'] = game_id
            team_df = team_df[available].copy()
            
            # Aggregate numeric columns if duplicate TEAM_ID + GAME_ID exists
            numeric_cols = team_df.select_dtypes(include='number').columns.tolist()
            team_df = team_df.groupby(['GAME_ID','TEAM_ID','TEAM_ABBREVIATION'], as_index=False)[numeric_cols].sum()
            
            return team_df
        except Exception as e:
            attempt += 1
            log(f"[WARN] GAME_ID {game_id}: {e}. Retry {attempt}/{MAX_RETRIES}")
            time.sleep(sleep_time)
            sleep_time *= 2  # Exponential backoff
    raise RuntimeError(f"Game {game_id}: boxscore could not be fetched.")

# ---------------- Main Flow ----------------

# Fetch all basic games for the season
all_games = fetch_basic_games()

# Normalize and calculate opponent points, point difference
basic = normalize_basic(all_games)

# Get unique game IDs and limit to last BATCH_SIZE games
unique_games = basic['GAME_ID'].drop_duplicates().tolist()
last_games = unique_games[-BATCH_SIZE:]

# Loop through the last games and insert boxscore into DB
for gid in tqdm(last_games, desc="Fetching & inserting team boxscores"):
    time.sleep(SLEEP_BETWEEN_CALLS)  # Throttle API calls
    try:
        team_stats = fetch_team_boxscore(gid)  # Fetch boxscore for this game
        for _, row in team_stats.iterrows():
            upsert_row(row)                      # Insert/Update DB
    except Exception as e:
        log(f"[WARN] GAME_ID {gid} could not be fetched: {e}")

log("[✓] Last 20 games team boxscores written to DB.")

[1] Sezon verisi çekiliyor: 2024-25 ...
[1] Toplam satır (takım-maç): 2802


Fetching boxscores: 100%|██████████| 20/20 [00:31<00:00,  1.57s/it]

[✓] CSV kaydedildi: nba_last20_boxscores.csv
